# Top Popular Recommender

In [ ]:
def init_kaggle(run_on_kaggle, secret_name):
    if run_on_kaggle:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        personal_token = user_secrets.get_secret(secret_name)
        
        import subprocess
        subprocess.run(["git", "clone", "https://" + personal_token + "@github.com/alecontuIT/rec_sys_challenge2022.git"])
        
        import os
        os.chdir("./rec_sys_challenge2022")
        
        subprocess.run(["pip", "install", "-r", "requirements.txt"])
        return True
    
    else:
        return False

In [ ]:
kaggle = init_kaggle(False, "recsys_git_token")

In [ ]:
import utils 
import os
from recmodels import TopPopRec
from Evaluation.Evaluator import EvaluatorHoldout
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Evaluation.Evaluator import EvaluatorHoldout

In [ ]:
dataset_version = "interactions-all-ones"
recommender_class = TopPopRec

In [ ]:
URM_csr, URM_train, URM_validation, _ = utils.get_data_global_sample(dataset_version=dataset_version, 
                                                                     train_percentage=0.7, 
                                                                     setSeed=True)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

## Validation

In [ ]:
hyperparameters_range_dictionary = {}

In [ ]:
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},
)

In [ ]:
utils.bayesian_search(
    recommender_class, 
    recommender_input_args, 
    hyperparameters_range_dictionary, 
    evaluator_validation,
    dataset_version=dataset_version,
    n_cases = 100,
    perc_random_starts = 0.3
)

## Best Model

In [ ]:
recommender = utils.fit_best_recommender(recommender_class, URM_csr, dataset_version)
utils.submission(recommender, dataset_version)

### Save Scores

In [ ]:
utils.save_item_scores(recommender_class, 
                       URM_train, 
                       evaluator_validation.users_to_evaluate, 
                       dataset_version, 
                       fast=True)